In [88]:
import folium
import re
import fileinput
import pandas as pd
from folium import IFrame

In [89]:
VEGAS_COORDINATES = (36.1699, -115.1398)
map = folium.Map(location=VEGAS_COORDINATES, zoom_start=11)

In [90]:
pos_df = pd.read_csv('pos_bsns.csv')
neg_df = pd.read_csv('neg_bsns.csv')
del(pos_df['Unnamed: 0'])
del(neg_df['Unnamed: 0'])

In [91]:
pos_topics = {0:'menu/experience', 1:'burger/fries', 2:'service/atmosphere', 
              3:'breakfast/pancakes', 4:'chicken/waffles', 5:'location/atmosphere'}

neg_topics = {0:'wait/table', 1:'burger/fries', 2:'bad_service', 
              3:'chicken/salad', 4:'breakfast/eggs', 5:'buffet/price'}

In [98]:
def star_color(stars):
    if stars < 2.0:
        return 'darkred'
    elif stars < 3.0:
        return 'lightred'
    elif stars < 4.0:
        return 'orange'
    elif stars < 5.0:
        return 'lightgreen'
    else:
        return 'darkgreen'

def get_pos_topics(id):
    positive = pos_df.loc[pos_df['business_id'] == "-CQokjildrY7UZezXCdEBw"]
    pos_scores = [positive['0'].get(0), positive['1'].get(0), positive['2'].get(0), 
                  positive['3'].get(0), positive['4'].get(0), positive['5'].get(0)]
    return pos_scores

def html_text(business_id, stars, neg_scores, name):
    html_topics1 = ['','','','','','']
    html_topics2 = ['','','','','','']
    
    pos_scores = get_pos_topics(business_id)
    for m in pos_topics:
        html_topics1[m] = 'Topic {}: {:18} {:.2f}%'.format(m, pos_topics[m], pos_scores[m]*100)
    for n in neg_topics:
        html_topics2[n] = 'Topic {}: {:18} {:.2f}%'.format(n, neg_topics[n], neg_scores[n]*100)
        
    pos_topics_text = 'Positive Topics<br>' + '<br>'.join(html_topics1)
    neg_topics_text = 'Negative Topics<br>' + '<br>'.join(html_topics2)
    stars_text = '<h3> Stars: ' + str(stars) + '</h3>'
    name_text = '<h3> Name: ' + name + '</h3>'
    return name_text + stars_text + '<p>' + pos_topics_text + neg_topics_text + '</p>'

def get_popup(business_id, stars, neg_scores, name):
    html = "" + html_text(business_id, stars, neg_scores, name) + ""
    iframe = folium.IFrame(html=html, width=300, height=200)
    return folium.Popup(iframe, max_width=1000)

In [99]:
for row in neg_df.itertuples():
    #print(row)
    probs = [row._7, row._8, row._9, row._10, row._11, row._12]
    folium.Marker(location=[row.latitude, row.longitude], popup=get_popup(row.business_id, row.stars_y, probs, row.name), 
                  icon=folium.Icon(color=star_color(row.stars_y), prefix='fa', icon='circle')).add_to(map)
    if row.Index > 100:
        break

In [94]:
legend_html = '''
    <div style=”position: fixed; 
    bottom: 50px; left: 50px; width: 100px; height: 90px; 
    border:2px solid grey; z-index:9999; font-size:14px;
    “>&nbsp; Legend <br>
    &nbsp; 0-2.0 &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:darkred"></i><br>
    &nbsp; 2.0-3.0 &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:lightred"></i>
    &nbsp; 3.0-4.0 &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:orange"></i>
    &nbsp; 4.0-5.0 &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:lightgreen"></i>
    &nbsp; 5.0 &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:darkgreen"></i>
    </div>
    '''
map.get_root().html.add_child(folium.Element(legend_html))

In [100]:
display(map)

In [96]:
map.save('vegas.html')

In [97]:
with open("vegas.html") as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for linenum,line in enumerate( fileinput.FileInput("vegas.html",inplace=1) ):
   pattern = markers[0] + ".bindPopup"
   pattern2 = markers[0] + ".on('mouseover', function (e) {this.openPopup();});"
   pattern3 = markers[0] + ".on('mouseout', function (e) {this.closePopup();});"

   if pattern in line:
      print(line.rstrip())
      print(pattern2)
      print(pattern3)
   else:
      print(line.rstrip())